In [1]:
from __future__ import annotations

import json
import cProfile
import pstats

In [2]:
from py_research.db import (
    DataSource,
    RecMap,
    RelMap,
    SubMap,
    SelIndex,
    DB
)

In [3]:
from test_db_schema import Search, Project, Task, User, Membership, Organization

In [4]:
data_source = DataSource(
    rec=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": RelMap(
            rel=Search.results,
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": RelMap(
                    rel=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": RelMap(
                            rel=Task.assignees,
                            push=User.name,
                            match=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": RelMap(
                    rel=Project.members,
                    push={User.name, User.age},
                    link=RecMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [5]:
with open("./nested_data.json") as f:
    data = json.load(f)

In [6]:
with cProfile.Profile() as pr:
  db = DB()
  rec = await data_source.load(data, db=db)
  
  stats = pstats.Stats(pr)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 with cProfile.Profile() as pr:                                                               │
│   2   db = DB()                                                                                  │
│ ❱ 3   rec = await data_source.load(data, db=db)                                                  │
│   4                                                                                              │
│   5   stats = pstats.Stats(pr)                                                                   │
│   6                                                                                              │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\data_source.py:583 in load                   │
│                                                                                                  │
│   580 │   │   Returns:                                                                           │
│   581 │   │   │   A Record instance                                                              │
│   582 │   │   """                                                                                │
│ ❱ 583 │   │   return await _load_record(self.rec, self, data, self._obj_cache, db)               │
│   584                                                                                            │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\data_source.py:548 in _load_record           │
│                                                                                                  │
│   545 │   │   │   │   rec_collection[idx] = target_rec                                           │
│   546 │   │                                                                                      │
│   547 │   │   if rec_collection is not None:                                                     │
│ ❱ 548 │   │   │   setattr(rec, rel.name, rec_collection)                                         │
│   549 │                                                                                          │
│   550 │   obj_cache[rec_type][rec._index] = rec                                                  │
│   551                                                                                            │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\base.py:3619 in __set__                      │
│                                                                                                  │
│   3616 │   ) -> None:                                                                            │
│   3617 │   │   if value is not State.undef:                                                      │
│   3618 │   │   │   rec = cast(Record[Hashable], instance)                                        │
│ ❱ 3619 │   │   │   instance._db[type(rec)][rec._index][self]._mutate(value)                      │
│   3620 │                                                                                         │
│   3621 │   @staticmethod                                                                         │
│   3622 │   def _get_tag(rec_type: type[Record]) -> RelSet | None:                                │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\base.py:3145 in _mutate                      │
│                                                            

In [ ]:
rec

In [ ]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")